# CIFAR-10 classifier from nnBuilder

This is a simple implementation of a CIFAR-10 classifier using the nnBuilder framework. Trying to reproduce the network of https://www.tensorflow.org/tutorials/deep_cnn/ (Work in progress)

In [ ]:
import tensorflow as tf
import numpy as np
import sys, time
sys.path.append('../nnBuilder')
from nnLayer import *
from nnInput import *
from nnTrainer import *
from nnHandler import *
from _nnUtils import *

In [ ]:
data=Layer(x=None,type="CIFAR_10")
preprocess_layers=[]
preprocess_layers.append(dict(type="Random_Crop",shape=[1,24,24,3]))
preprocess_layers.append(dict(type="Reshape",shape=[24,24,3]))
preprocess_layers.append(dict(type="Random_Horizontal_Flip"))
preprocess_layers.append(dict(type="Random_Brightness"))
preprocess_layers.append(dict(type="Random_Contrast"))
preprocess_layers.append(dict(type="Whitening"))
preprocess_layers_batch=[layer.copy() for layer in preprocess_layers if layer["type"]!="Reshape"]
for layer in preprocess_layers_batch:
    layer["type"]="Batch_"+layer["type"]
preprocess_layers_batch[0]["shape"]=preprocess_layers_batch[0]["shape"][1:]
#No preprocessing (40ms/batch, 95% gpu, 25% cpu)
batch_def_0=dict(x=data,type="Batch_Slice",batch=128)
#No input pipeline (200ms/batch, 30% gpu, 35% cpu)
batch_def_1=dict(x=data,type="Batch_Slice",batch=128,out_features=preprocess_layers_batch)
#With input pipeline and custom batch maker (memory problem if more than 1 thread, fixed?)
#1 Thread: (200ms/batch, 30% gpu, 35% cpu)
#2 Threads: (180ms/batch, ~40% gpu, 40% cpu)
#3 Threads: (175ms/batch, ~45% gpu, 45% cpu)
#Stopped Threads: (13ms/batch, ~90% gpu, 20% cpu)
batch_def_2=dict(x=data,type="Batch_Slice",batch=128,out_features=preprocess_layers_batch
                +[dict(type="Pipeline",num_threads=8)])
#With input pipeline and custom batch maker (quick memory fix, not working)
batch_def_3=dict(x=data,type="Batch_Slice",batch=128,in_features=[dict(type="Pipeline",num_threads=1)],
                 out_features=preprocess_layers_batch+[dict(type="Pipeline",num_threads=1)])
#With input pipeline and tensorflow'batch maker (labels broken)
batch_def_4=dict(x=data,type="Pipeline",batch=128,make_batch=True,shuffle=True,in_features=preprocess_layers)

batch=Layer(**batch_def_2)
layers=[]
layers.append(dict(type="Convolution",pad="SAME",window=5,stride=1,size=64,relu=True,out_features=[
              dict(type="Pool",pad="SAME",window=3,stride=2,pool_type="max"),
              dict(type="Local_Response_Norm")]))
layers.append(dict(type="Convolution",pad="SAME",window=5,stride=1,size=64,relu=True,out_features=[
              dict(type="Pool",pad="SAME",window=3,stride=2,pool_type="max"),
              dict(type="Local_Response_Norm")]))
layers.append(dict(type="Relu",size=384))
layers.append(dict(type="Relu",size=192))
layers.append(dict(type="Linear",size=10,in_features=["Dropout"]))
network_def=dict(type="Network",layers=layers)
network=Layer(x=batch,**network_def)
trainer=ClassifierTrainer(network=network,optimizer="adam",finish=True)
sess=SessManager(data,batch,network,trainer)#,input_network)
sess.start()

In [ ]:
trainer.train(0,keep_rate=1.)
trainer.eval_error()
for i in range(5):
    t0=time.time()
    trainer.train(100,keep_rate=1.)
    trainer.train(0,keep_rate=1.)
    trainer.eval_error() #Training error, reaches about 10%
    print("Time: %s"%(time.time()-t0))

In [ ]:
network.save() #The full network definition
save=network.save() #The full network definition
assert(Layer(x=batch,**save).save()==save) #Consistency
print(save==network_def) #Saving adds stuff, should still be equivalent
save

In [ ]:
sess=tf.InteractiveSession()

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction=0.
sess=tf.InteractiveSession(config=config)

In [ ]:
data=Layer(x=None,type="CIFAR_10")
preprocess_layers=[]
preprocess_layers.append(dict(type="Random_Crop",shape=[1,24,24,3]))
preprocess_layers.append(dict(type="Reshape",shape=[24,24,3]))
preprocess_layers.append(dict(type="Random_Horizontal_Flip"))
preprocess_layers.append(dict(type="Random_Brightness"))
preprocess_layers.append(dict(type="Random_Contrast"))
preprocess_layers.append(dict(type="Whitening"))
preprocess_layers_batch=[layer.copy() for layer in preprocess_layers if layer["type"]!="Reshape"]
for layer in preprocess_layers_batch:
    layer["type"]="Batch_"+layer["type"]
preprocess_layers_batch[0]["shape"]=preprocess_layers_batch[0]["shape"][1:]
#No preprocessing (40ms/batch, 95% gpu, 25% cpu)
batch_def_0=dict(x=data,type="Batch_Slice",batch=128)
#No input pipeline (200ms/batch, 30% gpu, 35% cpu)
batch_def_1=dict(x=data,type="Batch_Slice",batch=128,out_features=preprocess_layers_batch)
#With input pipeline and custom batch maker (memory problem if more than 1 thread)
#1 Thread: (200ms/batch, 30% gpu, 35% cpu)
#2 Threads: (180ms/batch, ~40% gpu, 40% cpu)
#3 Threads: (175ms/batch, ~45% gpu, 45% cpu)
#4+ Threads: OOM
#Stopped Threads: (13ms/batch, ~90% gpu, 20% cpu)
batch_def_2=dict(x=data,type="Batch_Slice",batch=128,out_features=preprocess_layers_batch
                +[dict(type="Pipeline",num_threads=1)])
#With input pipeline and custom batch maker (quick memory fix, not working)
batch_def_3=dict(x=data,type="Batch_Slice",batch=128,in_features=[dict(type="Pipeline",num_threads=1)],
                 out_features=preprocess_layers_batch+[dict(type="Pipeline",num_threads=1)])
#With input pipeline and tensorflow'batch maker (labels broken)
batch_def_4=dict(x=data,type="Pipeline",batch=128,make_batch=True,shuffle=True,in_features=preprocess_layers)

#batch=Layer(**batch_def_1)
layers=[]
layers.append(dict(type="Convolution",pad="SAME",window=5,stride=1,size=64,relu=True,out_features=[
              dict(type="Pool",pad="SAME",window=3,stride=2,pool_type="max"),
              dict(type="Local_Response_Norm")]))
layers.append(dict(type="Convolution",pad="SAME",window=5,stride=1,size=64,relu=True,out_features=[
              dict(type="Pool",pad="SAME",window=3,stride=2,pool_type="max"),
              dict(type="Local_Response_Norm")]))
layers.append(dict(type="Relu",size=384))
layers.append(dict(type="Relu",size=192))
layers.append(dict(type="Linear",size=10,in_features=["Dropout"]))
network_def=dict(type="Network",layers=layers)
#network=Layer(x=batch,**network_def)
#trainer=ClassifierTrainer(network=network,optimizer="adam",finish=True)
sess=SessManager(data)#,batch,network,trainer)#,input_network)
sess.start()

In [ ]:
var_sizes = [np.product(list(map(int, v.get_shape())))*v.dtype.size
             for key in data.get().graph.get_all_collection_keys() for v in data.get().graph.get_collection_ref(key)]
print(sum(var_sizes)/(1024**2), 'MB')

In [ ]:
data.get().graph.get_all_collection_keys()

In [ ]:
data.get().graph.get_collection_ref('variables')

In [ ]:
graph=tf.get_default_graph()
ops=graph.get_operations()
tensors=[t for op in ops for t in op.outputs]

In [ ]:
for tensor in tensors:
    print(np.product(tensor.get_shape().as_list()))

In [ ]:
for tensor in tensors:
    try:
        print(sess.run(tf.shape(tensor)))
    except:
        pass

In [ ]:
tensors[30]

In [ ]:
print(sess.run(tf.shape(tensors[30])))

In [ ]:
y=tf.Variable(tf.zeros(shape=[250000000], dtype=tf.float32))
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess=tf.InteractiveSession(config=config)
tf.global_variables_initializer().run()

In [ ]:
250000000*4/1024**2

In [ ]:
(2506-350)-953*2

In [ ]:
y=tf.reshape(tf.Variable(0, dtype=tf.float32,validate_shape=False),[250000000])
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess=tf.InteractiveSession(config=config)
tf.global_variables_initializer().run()

In [ ]:
import tensorflow as tf
import numpy as np
import sys, time
sys.path.append('../nnBuilder')
from nnLayer import *
from nnInput import *
from nnTrainer import *
from nnHandler import *
from _nnUtils import *

In [ ]:
a=np.ones([1000],dtype=np.float32)

In [ ]:
reader=tf.read_file("file")

In [ ]:
sess=tf.InteractiveSession()

In [ ]:
sess.run(tf.parse_tensor(reader, tf.float32))

In [ ]:
#sess.run(reader)

In [ ]:
#sess.run(tf.parse_tensor(reader, tf.float32))

In [ ]:
a=np.ones(1,dtype=np.float32).tostring()
b=tf.constant(a)
sess.run(b)
#sess.run(tf.parse_tensor(b, tf.float32))

In [ ]:
reader=tf.read_file("file")
b=sess.run(tf.decode_raw(reader, tf.float32))[3:-1]

In [ ]:
a=np.ones([25000000],dtype=np.float32)
writer=tf.python_io.TFRecordWriter("file")
writer.write(a.tostring())
writer.close()

In [ ]:
reader=tf.read_file("file")
decode=tf.decode_raw(reader, tf.float32)[3:-1]
var=tf.Variable(decode,validate_shape=False)

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess=tf.InteractiveSession(config=config)
tf.global_variables_initializer().run()

In [ ]:
sess.run(decode)

In [ ]:
data.run().shape